## Checkpoints
- [ ] ``load -> save -> compare``
- [ ] ``make pkg from scratch``
- [ ] ``layers generator``

In [30]:
import os

conda_env = bool(os.getenv('CONDA_DEFAULT_ENV')) or bool(os.getenv('CONDA_PYTHON_EXE'))  or bool(os.getenv('CONDA_DIR'))
print(conda_env)

True


In [ ]:
upgrade_py = True
if upgrade_py:
    if conda_env:
        !conda install -y python-dotenv
    else:
        !python3.10 -m pip install --upgrade pip
        !pip install graph-notebook
        !pip install graphviz
        !pip install pydot
        !pip install networkx
        !pip install pyvis
        !pip install platform
        !pip install hashlib
        !pip install jycm
        !pip install pipreqs
%load_ext line_profiler

Channels:
 - defaults
 - conda-forge
 - pytorch
 - nvidia
 - anaconda
Platform: linux-64
- llecting package metadata (repodata.json): - 

ERROR: Could not find a version that satisfies the requirement platform (from versions: none)
ERROR: No matching distribution found for platform
ERROR: Ignored the following yanked versions: 20081119
ERROR: Could not find a version that satisfies the requirement hashlib (from versions: none)
ERROR: No matching distribution found for hashlib
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import networkx as nx
import json
import pprint
import numpy as np
import pandas as pd
import time
import platform
import hashlib
from enum import Enum
from pyvis.network import Network
from IPython.display import display, HTML
import copy
import tqdm
from jycm.jycm import YouchamaJsonDiffer
from IPython.display import JSON
from IPython.display import Markdown

from pathlib import Path
os.environ["MGENN_DEBUG"] = "Y"


In [ ]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(Path(os.getcwd()).parent.absolute())

import mcore as mc

from common import *

In [ ]:
pkg = mc.Package()
pkg.loadFile('data/namespacetest_rev0.pkg')
pkg.counts()


In [ ]:
def showGraph(pkg):
    #net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed =True)

    ZH = "ZERO_HOLE"
    def hasNode(nodes, key)->bool:
        for node in nx_graph.nodes:
            if str(node) == str(key):
                return True
        return False
    nx_graph = nx.DiGraph()

    for n in pkg.neurons:
        nx_graph.add_node(str(n["id"]), size = 10 + int(float(n["currentEnergy"])), title=('neuron %d' % n["id"]), group=1)
    for o in pkg.outputs:
        nx_graph.add_node(str(o["id"]), size = 10, title=('outputs %d' % n["id"]), group=2)
    for i in pkg.inputs:
        nx_graph.add_node(str(i["name"]), size = 5, title=('input %s' % i["name"]), group=3)

    #make direct links
    nx_graph.add_node(str(ZH), size = 1, title=("missed node"), group=1)
    l = pkg.links[1]
    start, stop = pkg.linkEnds(l)
    #print("link ", start, " to " , stop)
    #nx_graph.add_edge(str(start), str(stop))

    for l in pkg.links:
        print(l)
        start, stop = pkg.linkEnds(l)
        link_len = int(l['length'])
        print("link ", start, " to " , str(stop), " sz:", link_len)
        if not start:
            continue
            nx_graph.add_edge(ZH, stop, length = (link_len + 1), weight=(link_len + 1))
        if not hasNode(nx_graph.nodes, str(stop)):
            continue
            nx_graph.add_edge(ZH, stop, value = (link_len + 1), weight=(link_len + 1))
        nx_graph.add_edge(str(start), str(stop), length = (link_len + 1), weight=(link_len + 1) )

    for n in pkg.neurons:
        for rc in n['receivers']:
            if not pkg.isLink(rc):
                nx_graph.add_edge(str(n["id"]), str(rc), length = 1, weight = 1)
    nt.from_nx(nx_graph)
    return nt.show('nx.html')

In [ ]:
showGraph(pkg)

## low-level generators

In [ ]:


p = pkg.clone()
n1 = ObjectFactory.makeNeuronData(peak = 10.0, receivers = [], leak = 0.1, currentEnergy = 3.0, id = p.nextId())
p.neurons.append(n1)
p.connect(n1["id"], 64)
p.connect(n1["id"], 65)
p.connect(n1["id"], 69)
g = showGraph(p)
print("new object",n1["id"])
g
pkg = None

## high-level generators

In [ ]:
pkg = mc.Package.make_empty()
n1 = pkg.new_neuron(leak=1.0, peak=5.0, receivers=[])
i1 = pkg.new_tape_input("i1", [])
o1 = pkg.new_output("o1")

l1 = pkg.new_link_between(apt=0.0, length=1, src=i1, dst=n1)
l2 = pkg.new_link_between(apt=0.0, length=1, src=n1, dst=o1)
print(f"i_{i1} --[{l1}]-> N_{n1} --[{l2}]-> o_{o1}")
showGraph(pkg)

# simple network

In [ ]:
F.make_quiet()
pkg = mc.Package.make_empty()
n1 = pkg.new_neuron(leak=1.0, peak=5.0, receivers=[])
i1 = pkg.new_tape_input("i1", [])
o1 = pkg.new_output("o1")

l1 = pkg.new_link_between(apt=0.0, length=1, src=i1, dst=n1)
l2 = pkg.new_link_between(apt=0.0, length=1, src=n1, dst=o1)
ce = mc.Engine()
ce.core = mc.Core()
ce.core.load(pkg)
ce.tick_offset = 0

df_expected = pd.DataFrame([0.0]*7 + [5.0] + [0.0]*4, columns=['o1_exp'])
df_in = pd.DataFrame([3,0] * 6, columns=['i1'])
out = [0] * 9 + [5, 0]
ticks = 12
df_out = pd.DataFrame()
for t in range(ticks):
    df_out = pd.concat([df_out, ce.run_once(df_in)], ignore_index=True)



[2024-12-27 15:36:49]in[__process_autoinputs.153] autoinputs made 0 new events
[2024-12-27 15:36:49]in[dprint.58] TapeInputsRow h[1], d[s1.dim1]
[2024-12-27 15:36:49]in[makeEvents.115] proces point [i1] : ([3], {})
[2024-12-27 15:36:49]in[dprint.58] TapeInputsRow h[1], d[s1.dim1]
[2024-12-27 15:36:49]in[__process_tapes.160] tape container made 1 new events
[2024-12-27 15:36:49]in[__on_e_changed.85] N[1] changed 2.0 --> 1.0 in [onTick.82]
[2024-12-27 15:36:49]in[onRobotsEvent.31] @ Link.3.bfc4c7ec7e14ef53 LINK_APPLY_EVENT : {'tick': 10, 'amp': 3.0, 'from': 'i1'}
[2024-12-27 15:36:49]in[__process_content.166] active content (4 objects) made 1 new events
[2024-12-27 15:36:49]in[__process_events.177] process p.event i1->Link_3 amp:0
[2024-12-27 15:36:49]in[onRobotsEvent.31] @ Link.3.764e00faf6fb076a LINK_IGNORE_SIGNAL : {'tick': 10, 'amp': 0, 'from': 'i1'}
[2024-12-27 15:36:49]in[__process_events.177] process p.event 3->Neuron_1 amp:3.0
[2024-12-27 15:36:49]in[onRobotsEvent.31] @ Neuron.1.

In [ ]:
io = pd.concat([df_in, df_out, df_expected], axis=1)
io['error'] = io['o1'].astype(float) - io['o1_exp'].astype(float)
io

# Worm with breakpoints
## make worm

In [ ]:
F.make_quiet()
#F.make_verbose()
pkg = mc.Package.make_empty()
o_end = pkg.new_output("o_end")
w_size = 10
last_o = o_end
last_n = None
ns = []
for i in range(w_size):
    ni = pkg.new_neuron(leak=0.0, peak=5.0, receivers=[])
    pkg.new_link_between(apt=0.0, length=1, src=ni, dst=last_o)
    last_o = pkg.new_output(f"o_end-{i}")
    ns.append(ni)
    if last_n:
        pkg.new_link_between(apt=1.0, length=1, src=ni, dst=last_n)
    last_n = ni
i1 = pkg.new_tape_input("i1", [])
pkg.new_link_between(apt=0.0, length=1, src=i1, dst=last_n)
showGraph(pkg)


## run worm

In [ ]:

ce = None
dirty_pkg = pkg.clone()
ce = mc.Engine()
ce.core = mc.Core()
ce.core.load(dirty_pkg)
ce.tick_offset = 0
ticks = 200
df_expected = pd.DataFrame([1.0]*(20) + [0.0]*(ticks-20), columns=['o_end'])
df_in = pd.DataFrame([1.0]*(20) + [0.0]*(ticks-20), columns=['i1'])
total_energy = []

df_out = pd.DataFrame()
for t in tqdm.tqdm(range(ticks)):
    df_out = pd.concat([df_out, ce.run_once(df_in)], ignore_index=True)
    dpkg = ce.core.dump()
    energy = dpkg.total_energy()
    total_energy.append(energy)
e_df = pd.DataFrame(total_energy, columns=["E"])
ax = e_df.plot(kind='line', title='total E', grid='on')
plt.xlabel('tick')
plt.ylabel('Energy')
plt.show()



## bars output
ax = df_out.plot(kind='bar', title='Outputs B', grid='on')
plt.xlabel('tick')
plt.ylabel('Energy')
plt.show()

#line output
ax = df_out.plot(kind='line', title='Outputs L', grid='on')
plt.xlabel('tick')
plt.ylabel('Energy')
plt.show()

In [ ]:
df_out